# Merged codebase for the entire ML/Computer Vision parts of the project

###### *This file needs to be placed on ../Backend/*

### Imports packages needed and defines GLOBAL variables

In [1]:
# All imports needed
from ultralytics import YOLO
from PIL import Image
import csv
from RPi.utils import * # GLOBAL Parameters defined here

if not os.path.isdir(tmp_dir):
    os.makedirs(tmp_dir)

### Loads pre-trained models

In [2]:
cropResistor = YOLO("YOLO_Cropper\\latest\\best.pt")
findColorBands = YOLO("YOLO_Segmenter\\latest\\best_v11.pt")

### Code used for extracting and handling color classes

In [3]:
# Extracts the CSV files' color data to create color classes
def extract_colors(directory):
    classes = dict()
    for file in os.listdir(directory):
        if os.path.isfile(file):
            ext = file.split('.')[-1]
            name = ".".join(file.split('.')[:-1])
            if ext == "csv":
                with open(file) as csv_file:
                    csv_reader = csv.reader(csv_file, delimiter=',')
                    line_count = 0
                    summ = np.array([0,0,0])
                    for row in csv_reader:
                        H = int(row[0])
                        S = int(row[1])
                        V = int(row[2])
                        hsv = np.array([H,S,V])
                        line_count += 1
                        summ += hsv
                
                classes[name] = summ/line_count
    return classes

### Merged inference code

In [7]:
file = "Functionality Tests\\black_bg_02.jpg"

In [8]:
# Runs crop detection model on file
cropped_inf = cropResistor(file, conf=0.6)

# Crops and saves original file to a temporary location
if len(cropped_inf[0].boxes.data):
    cropped = cropImage(cv.imread(file), cropped_inf[0].boxes.cpu().data[0])
else:
    cropped = cv.imread(file)
Image.fromarray(cv.cvtColor(cropped, cv.COLOR_BGR2RGB)).save(tmp_crop)


image 1/1 d:\Joule\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\Functionality Tests\black_bg_02.jpg: 640x480 1 Resistor, 19.0ms
Speed: 10.5ms preprocess, 19.0ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 480)


In [9]:
# Runs color bands segmentation model on cropped inference
if len(cropped_inf[0].boxes.data):
    colorbands_inf = findColorBands(tmp_crop, save=True, conf=0.6, iou=0.3)
else:
    colorbands_inf = findColorBands(file, save=True, conf=0.6, iou=0.3)


image 1/1 d:\Joule\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\tmp\crop.png: 576x640 3 Color-Bands, 64.1ms
Speed: 9.5ms preprocess, 64.1ms inference, 4.0ms postprocess per image at shape (1, 3, 576, 640)
Results saved to runs\segment\predict22


### Finds the order of bands

In [ ]:
# Gets masks objects from the inference
masks, _ = timer(get_segmentation_masks, tmp_crop, colorbands_inf, printout=True)

# Sorts the masks by distance (it doesn't have a way to 
#    know in which direction to start, so by default it 
#    uses the leftmost (0,0) point as a beginning.
ordered, _ = timer(order_masks, masks, colorbands_inf, printout=True)

In [ ]:
for i in range(len(ordered)):
    print(f"BBOX: {[int(j) for j in ordered[i].bbox[:4]]} \t Index: {ordered[i].index} \t HSV: {cvtBGR2HSV(ordered[i].avgColor, paint=True)}")

In [ ]:
# Retrieves the color name from masks average color
# Numbers on the side represent the number of samples used for each color's average
#  Files used to sample: res2.png, res.png, crop.jpg
COLOR_RANGES = {}
def sample_color_names(mask):
    pass